d ##### Usecase:Get sales data by departments
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for departments per day
###### Problem statement-2:Get sales data for departments per month
###### Problem statement-3:Get sales data for departments per year
###### Problem statement-4:Get monthly sales data from TOP5 revenue generatin departments
###### Problem statement-5:Get third and final quarters of 2013 sales data from TOP5 revenue generating products for each month with rankings
###### Problem statement-6:Get sales data for Men's product
###### Problem statement-7:Get monthly sales data from kids and women categories for first two quarters of 2014 
###### Problem statement-8::Get monthly sales data of TOP10 revenue generating Nike and adidas products with rankings
###### Problem statement-9:Get quarterly sales data of golf products for 2013 final quarter
###### Problem statement-10:Get sales data of TOP3 Apparel products for each year

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('SalesDatabydepartments').enableHiveSupport().getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions",10)

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
customerItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Customers.txt", sep=';', schema='customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, customer_state string, customer_zipcode string')
customerItemsdf.show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [6]:
productItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Products.txt", sep=';', schema='product_id int, product_category_id int, product_name string, product_desc string, product_price float, product_image string ')
productItemsdf.show(5,False)

+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
product_id|product_category_id|product_name |product_desc|product_price|product_image |
+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
1 |2 |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|null |59.98 |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy|
2 |2 |Under Armour Men's Highlight MC Football Clea|null |129.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat |
3 |2 |Under Armour Men's Renegade D Mid Football Cl|null |89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat |
4 |2 |Under Armour Men's Renegade D Mid Football Cl|null |89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat |
5 |2 |Riddell Youth Revolution Speed Custom Footbal|null |199.99 |http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet|
+----------+-------------------+---------------------------------------------+------------+-------------+-------------------------------------------------------------------------------------+
only showing top 5 rows

In [7]:
categoriesdf = spark.read.csv("/FileStore/tables/retaildbtext/Categories.txt", sep=',', schema='category_id int, category_department_id  int, category_name string')
categoriesdf.show(5,False)

+-----------+----------------------+-------------------+
category_id|category_department_id|category_name |
+-----------+----------------------+-------------------+
1 |2 |Football |
2 |2 |Soccer |
3 |2 |Baseball & Softball|
4 |2 |Basketball |
5 |2 |Lacrosse |
+-----------+----------------------+-------------------+
only showing top 5 rows

In [8]:
departmentsdf = spark.read.csv("/FileStore/tables/retaildbtext/Departments.txt", sep=',', schema='department_id int, department_name string')
departmentsdf.show(5,False)

+-------------+---------------+
department_id|department_name|
+-------------+---------------+
2 |Fitness |
3 |Footwear |
4 |Apparel |
5 |Golf |
6 |Outdoors |
+-------------+---------------+
only showing top 5 rows

In [9]:
ordersMap = ordersdf.select('order_id','order_date','order_status')
ordersMap.show(5,False)

+--------+----------+---------------+
order_id|order_date|order_status |
+--------+----------+---------------+
1 |2013-07-25|CLOSED |
2 |2013-07-25|PENDING_PAYMENT|
3 |2013-07-25|COMPLETE |
4 |2013-07-25|CLOSED |
5 |2013-07-25|COMPLETE |
+--------+----------+---------------+
only showing top 5 rows

In [10]:
orderItemsMap=orderItemsdf.select('order_item_order_id','order_item_product_id','order_item_subtotal','order_item_quantity')
orderItemsMap.show(5,False)

+-------------------+---------------------+-------------------+-------------------+
order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+-------------------+---------------------+-------------------+-------------------+
1 |957 |299.98 |1 |
2 |1073 |199.99 |1 |
2 |502 |250.0 |5 |
2 |403 |129.99 |1 |
4 |897 |49.98 |2 |
+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [11]:
ordersdfjoin = ordersMap.join(orderItemsMap, ordersMap.order_id == orderItemsMap.order_item_order_id,'inner')
ordersdfjoin.show(5,False)


+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
order_id|order_date|order_status |order_item_order_id|order_item_product_id|order_item_subtotal|order_item_quantity|
+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
1 |2013-07-25|CLOSED |1 |957 |299.98 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |1073 |199.99 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |502 |250.0 |5 |
2 |2013-07-25|PENDING_PAYMENT|2 |403 |129.99 |1 |
4 |2013-07-25|CLOSED |4 |897 |49.98 |2 |
+--------+----------+---------------+-------------------+---------------------+-------------------+-------------------+
only showing top 5 rows

In [12]:
ordersdfSalesmap = ordersdfjoin.select('order_id','order_date','order_item_product_id','order_status','order_item_subtotal','order_item_quantity')
ordersdfSalesmap.show(10,False)

+--------+----------+---------------------+---------------+-------------------+-------------------+
order_id|order_date|order_item_product_id|order_status |order_item_subtotal|order_item_quantity|
+--------+----------+---------------------+---------------+-------------------+-------------------+
1 |2013-07-25|957 |CLOSED |299.98 |1 |
2 |2013-07-25|1073 |PENDING_PAYMENT|199.99 |1 |
2 |2013-07-25|502 |PENDING_PAYMENT|250.0 |5 |
2 |2013-07-25|403 |PENDING_PAYMENT|129.99 |1 |
4 |2013-07-25|897 |CLOSED |49.98 |2 |
4 |2013-07-25|365 |CLOSED |299.95 |5 |
4 |2013-07-25|502 |CLOSED |150.0 |3 |
4 |2013-07-25|1014 |CLOSED |199.92 |4 |
5 |2013-07-25|957 |COMPLETE |299.98 |1 |
5 |2013-07-25|365 |COMPLETE |299.95 |5 |
+--------+----------+---------------------+---------------+-------------------+-------------------+
only showing top 10 rows

In [13]:
productsMap = productItemsdf.select('product_id','product_category_id','product_name','product_price')
productsMap.show(5,False)

+----------+-------------------+---------------------------------------------+-------------+
product_id|product_category_id|product_name |product_price|
+----------+-------------------+---------------------------------------------+-------------+
1 |2 |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|59.98 |
2 |2 |Under Armour Men's Highlight MC Football Clea|129.99 |
3 |2 |Under Armour Men's Renegade D Mid Football Cl|89.99 |
4 |2 |Under Armour Men's Renegade D Mid Football Cl|89.99 |
5 |2 |Riddell Youth Revolution Speed Custom Footbal|199.99 |
+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [14]:
prodordersdfjoin = ordersdfSalesmap.join(productsMap, ordersdfSalesmap.order_item_product_id == productsMap.product_id,'inner')
prodordersdfjoin.show(5,False)

+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
order_id|order_date|order_item_product_id|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|
+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
1 |2013-07-25|957 |CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |
2 |2013-07-25|1073 |PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |
2 |2013-07-25|502 |PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |
2 |2013-07-25|403 |PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |
4 |2013-07-25|897 |CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |
+--------+----------+---------------------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [15]:
prodordersdfsalesmap =prodordersdfjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity','product_id','product_category_id','product_name','product_price')
prodordersdfsalesmap.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+
only showing top 5 rows

In [16]:
prodcatdfsalesjoin = prodordersdfsalesmap.join(categoriesdf, prodordersdfsalesmap.product_category_id == categoriesdf.category_id,'inner')
prodcatdfsalesjoin.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_id|category_department_id|category_name |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |43 |7 |Camping & Hiking|
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |48 |7 |Water Sports |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |24 |5 |Women's Apparel |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |18 |4 |Men's Footwear |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |40 |6 |Accessories |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
only showing top 5 rows

In [17]:
prodordersdfsalesmap=prodcatdfsalesjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity','product_id','product_category_id','product_name',\
                                               'product_price','category_id','category_department_id','category_name')
prodordersdfsalesmap.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_id|category_department_id|category_name |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |43 |7 |Camping & Hiking|
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |48 |7 |Water Sports |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |24 |5 |Women's Apparel |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |18 |4 |Men's Footwear |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |40 |6 |Accessories |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+
only showing top 5 rows

In [18]:
prodcatdfsalesjoin = prodordersdfsalesmap.join(departmentsdf, prodordersdfsalesmap.category_department_id == departmentsdf.department_id,'inner')
prodcatdfsalesjoin.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_id|category_department_id|category_name |department_id|department_name|
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |43 |7 |Camping & Hiking|7 |Fan Shop |
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |48 |7 |Water Sports |7 |Fan Shop |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |24 |5 |Women's Apparel |5 |Golf |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |18 |4 |Men's Footwear |4 |Apparel |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |40 |6 |Accessories |6 |Outdoors |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
only showing top 5 rows

In [19]:
deptprodordersdfsalesmap=prodcatdfsalesjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity','product_id','product_category_id','product_name',\
                                               'product_price','category_id','category_department_id','category_name','department_id','department_name')
deptprodordersdfsalesmap.show(5,False)

+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|product_id|product_category_id|product_name |product_price|category_id|category_department_id|category_name |department_id|department_name|
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
1 |2013-07-25|CLOSED |299.98 |1 |957 |43 |Diamondback Women's Serene Classic Comfort Bi|299.98 |43 |7 |Camping & Hiking|7 |Fan Shop |
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |1073 |48 |Pelican Sunstream 100 Kayak |199.99 |48 |7 |Water Sports |7 |Fan Shop |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |502 |24 |Nike Men's Dri-FIT Victory Golf Polo |50.0 |24 |5 |Women's Apparel |5 |Golf |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |403 |18 |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |18 |4 |Men's Footwear |4 |Apparel |
4 |2013-07-25|CLOSED |49.98 |2 |897 |40 |Team Golf New England Patriots Putter Grip |24.99 |40 |6 |Accessories |6 |Outdoors |
+--------+----------+---------------+-------------------+-------------------+----------+-------------------+---------------------------------------------+-------------+-----------+----------------------+----------------+-------------+---------------+
only showing top 5 rows

In [20]:
deptprodordersdfsalesmap.printSchema()

root
-- order_id: integer (nullable = true)
-- order_date: string (nullable = true)
-- order_status: string (nullable = true)
-- order_item_subtotal: float (nullable = true)
-- order_item_quantity: integer (nullable = true)
-- product_id: integer (nullable = true)
-- product_category_id: integer (nullable = true)
-- product_name: string (nullable = true)
-- product_price: float (nullable = true)
-- category_id: integer (nullable = true)
-- category_department_id: integer (nullable = true)
-- category_name: string (nullable = true)
-- department_id: integer (nullable = true)
-- department_name: string (nullable = true)

###### Problem statement-1:Get sales data for departments per day

In [22]:
departmentsSalesDataPerDay = deptprodordersdfsalesmap.groupBy('order_date','department_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('order_date','department_name')
departmentsSalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerDay", compression="none", header ='true')
departmentsSalesDataPerDay.show(20,False)

+----------+---------------+--------+---------+---------+----------+-------------+-------------+
order_date|department_name|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+---------------+--------+---------+---------+----------+-------------+-------------+
2013-07-25|Apparel |152.89 |59.99 |299.95 |13148.31 |169 |86 |
2013-07-25|Fan Shop |252.89 |49.98 |399.98 |34392.89 |227 |136 |
2013-07-25|Fitness |141.23 |69.98 |319.96 |564.91 |13 |4 |
2013-07-25|Footwear |322.59 |99.99 |599.99 |9355.06 |94 |29 |
2013-07-25|Golf |136.05 |39.99 |250.0 |8299.24 |178 |61 |
2013-07-25|Outdoors |104.06 |19.98 |239.97 |2393.42 |60 |23 |
2013-07-26|Apparel |160.03 |59.99 |299.95 |31525.69 |431 |197 |
2013-07-26|Fan Shop |260.57 |29.99 |495.0 |67227.82 |393 |258 |
2013-07-26|Fitness |61.08 |25.0 |124.95 |671.91 |25 |11 |
2013-07-26|Footwear |238.02 |30.0 |499.95 |15947.3 |176 |67 |
2013-07-26|Golf |138.82 |39.99 |280.0 |17213.49 |379 |124 |
2013-07-26|Outdoors |106.32 |9.99 |399.95 |3933.96 |107 |37 |
2013-07-27|Apparel |149.3 |59.99 |299.95 |17617.74 |226 |118 |
2013-07-27|Fan Shop |259.92 |49.98 |399.98 |56663.48 |354 |218 |
2013-07-27|Fitness |86.98 |24.99 |124.95 |347.94 |15 |4 |
2013-07-27|Footwear |281.92 |27.99 |499.95 |11276.91 |114 |40 |
2013-07-27|Golf |128.69 |30.0 |250.0 |12353.94 |272 |96 |
2013-07-27|Outdoors |104.23 |17.99 |260.0 |2814.33 |76 |27 |
2013-07-28|Apparel |163.58 |59.99 |299.95 |24536.75 |325 |150 |
2013-07-28|Fan Shop |265.67 |29.99 |399.98 |41975.36 |250 |158 |
+----------+---------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for departments per month

In [24]:
departmentsSalesDataPerMonth =deptprodordersdfsalesmap.groupBy(substring('order_date',0,7).alias("ordermonth"),'department_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('ordermonth','department_name')
departmentsSalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerMonth", compression="none", header ='true')
departmentsSalesDataPerMonth.show(20,False)

+----------+---------------+--------+---------+---------+----------+-------------+-------------+
ordermonth|department_name|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+----------+---------------+--------+---------+---------+----------+-------------+-------------+
2013-07 |Apparel |158.67 |59.99 |299.99 |163108.28 |2173 |1028 |
2013-07 |Fan Shop |261.21 |29.99 |495.0 |389462.08 |2442 |1491 |
2013-07 |Fitness |112.44 |22.0 |319.96 |5622.15 |138 |50 |
2013-07 |Footwear |263.63 |27.99 |599.99 |84624.92 |917 |321 |
2013-07 |Golf |139.82 |25.0 |350.0 |101087.26 |2190 |723 |
2013-07 |Outdoors |100.37 |9.99 |399.95 |20877.52 |561 |208 |
2013-08 |Apparel |157.25 |59.99 |299.99 |598981.23 |7879 |3809 |
2013-08 |Fan Shop |256.01 |29.99 |495.0 |1410128.67|8892 |5508 |
2013-08 |Fitness |137.11 |22.0 |499.95 |23719.61 |477 |173 |
2013-08 |Footwear |270.12 |27.99 |1999.99 |321443.79 |3486 |1190 |
2013-08 |Golf |139.83 |25.0 |350.0 |391535.73 |8447 |2800 |
2013-08 |Outdoors |105.14 |9.99 |500.0 |82849.72 |2130 |788 |
2013-09 |Apparel |156.24 |59.99 |299.99 |615739.57 |8045 |3941 |
2013-09 |Fan Shop |257.7 |29.99 |399.98 |1452927.71|8934 |5638 |
2013-09 |Fitness |130.36 |22.0 |999.99 |24377.45 |513 |187 |
2013-09 |Footwear |277.43 |27.99 |1999.99 |369264.45 |3993 |1331 |
2013-09 |Golf |137.98 |25.0 |350.0 |388137.44 |8373 |2813 |
2013-09 |Outdoors |107.25 |9.99 |500.0 |84080.7 |2289 |784 |
2013-10 |Apparel |155.57 |59.99 |299.99 |562546.12 |7390 |3616 |
2013-10 |Fan Shop |258.15 |29.99 |399.98 |1297740.37|8064 |5027 |
+----------+---------------+--------+---------+---------+----------+-------------+-------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for departments per year

In [26]:
departmentsSalesDataPerYear = deptprodordersdfsalesmap.groupBy(substring('order_date',0,4).alias("orderyear"),'department_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('orderyear','department_name')
departmentsSalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerYear", compression="none", header ='true')
departmentsSalesDataPerYear.show(20,False)

+---------+---------------+--------+---------+---------+----------+-------------+-------------+
orderyear|department_name|avgsales|min_sales|max_sales|Totalsales|Totalquantity|Numberofsales|
+---------+---------------+--------+---------+---------+----------+-------------+-------------+
2013 |Apparel |156.23 |59.99 |299.99 |3228557.64|42247 |20665 |
2013 |Fan Shop |258.11 |29.99 |495.0 |7615003.82|47141 |29503 |
2013 |Fitness |129.96 |22.0 |999.99 |123587.89 |2635 |951 |
2013 |Footwear |272.67 |27.99 |1999.99 |1789823.74|19514 |6564 |
2013 |Golf |138.64 |25.0 |350.0 |2058230.9 |44369 |14846 |
2013 |Outdoors |106.97 |9.99 |599.99 |438985.38 |11503 |4104 |
2014 |Apparel |156.71 |59.99 |299.99 |4095142.81|53733 |26132 |
2014 |Fan Shop |257.75 |29.99 |495.0 |9492762.46|58495 |36829 |
2014 |Fitness |133.61 |22.0 |999.99 |156456.25 |3235 |1171 |
2014 |Footwear |278.44 |27.99 |1999.99 |2216675.03|23886 |7961 |
2014 |Golf |138.83 |25.0 |350.0 |2550797.34|54928 |18374 |
2014 |Outdoors |109.18 |9.99 |599.99 |556597.34 |14072 |5098 |
+---------+---------------+--------+---------+---------+----------+-------------+-------------+

###### Problem statement-4:Get sales data from TOP5 reveue generating departments

In [28]:
departmentsSalesDatafortop5dept = deptprodordersdfsalesmap.groupBy('department_name').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("deptrevenue"),
                                                                sum('order_item_quantity').alias("Totalquantity"),
                                                                count('order_item_subtotal').alias("Numberofsales")).sort('deptrevenue',ascending=[0]).limit(5)
departmentsSalesDatafortop5dept.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDatafortop5dept", compression="none", header ='true')
departmentsSalesDatafortop5dept.show(5,False)

+---------------+--------+---------+---------+-------------+-------------+-------------+
department_name|avgsales|min_sales|max_sales|deptrevenue |Totalquantity|Numberofsales|
+---------------+--------+---------+---------+-------------+-------------+-------------+
Fan Shop |257.91 |29.99 |495.0 |1.710776628E7|105636 |66332 |
Apparel |156.5 |59.99 |299.99 |7323700.45 |95980 |46797 |
Golf |138.74 |25.0 |350.0 |4609028.24 |99297 |33220 |
Footwear |275.83 |27.99 |1999.99 |4006498.77 |43400 |14525 |
Outdoors |108.19 |9.99 |599.99 |995582.72 |25575 |9202 |
+---------------+--------+---------+---------+-------------+-------------+-------------+

###### Problem statement-5:Get third and final quarters of 2013 sales data from TOP5 revenue generating departments for each month with rankings

In [30]:
SalesDataPerMonth=deptprodordersdfsalesmap.where((to_date(deptprodordersdfsalesmap.order_date) >=lit("2013-07-01")) & 
                                                 (to_date(deptprodordersdfsalesmap.order_date) <=lit("2013-12-31")))


departmentSalesDatafortop5perquart= SalesDataPerMonth.groupBy(substring('order_date',1,7).alias('order_month'),'department_name').agg(round(avg('order_item_subtotal'),2).alias("avg_sales_permonth"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("deptrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth"))                                                 

window = Window.partitionBy(departmentSalesDatafortop5perquart['order_month']).orderBy(departmentSalesDatafortop5perquart['deptrevenue'].desc())
top5deptdataeachmonth=departmentSalesDatafortop5perquart.select('order_month','department_name','deptrevenue').withColumn('rank', rank().over(window).alias('rank'))
top5rankeddeptdataeachmonth = top5deptdataeachmonth.filter(col('rank')<=5).sort('order_month','deptrevenue',ascending=[1,0])
top5rankeddeptdataeachmonth.coalesce(1).write.csv("/FileStore/tables/top5rankedorderseachmonth", compression="none", header ='true')          
top5rankeddeptdataeachmonth.show(20,False)
                               

                                 

+-----------+---------------+-----------+----+
order_month|department_name|deptrevenue|rank|
+-----------+---------------+-----------+----+
2013-07 |Fan Shop |389462.08 |1 |
2013-07 |Apparel |163108.28 |2 |
2013-07 |Golf |101087.26 |3 |
2013-07 |Footwear |84624.92 |4 |
2013-07 |Outdoors |20877.52 |5 |
2013-08 |Fan Shop |1410128.67 |1 |
2013-08 |Apparel |598981.23 |2 |
2013-08 |Golf |391535.73 |3 |
2013-08 |Footwear |321443.79 |4 |
2013-08 |Outdoors |82849.72 |5 |
2013-09 |Fan Shop |1452927.71 |1 |
2013-09 |Apparel |615739.57 |2 |
2013-09 |Golf |388137.44 |3 |
2013-09 |Footwear |369264.45 |4 |
2013-09 |Outdoors |84080.7 |5 |
2013-10 |Fan Shop |1297740.37 |1 |
2013-10 |Apparel |562546.12 |2 |
2013-10 |Golf |362129.11 |3 |
2013-10 |Footwear |300716.9 |4 |
2013-10 |Outdoors |77542.62 |5 |
+-----------+---------------+-----------+----+
only showing top 20 rows

###### Problem statement-6:Get sales data for Men's product

In [32]:
departmentsSalesDataforMensprod= deptprodordersdfsalesmap.groupBy('department_name','product_name').agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("productrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter(deptprodordersdfsalesmap.product_name.contains("Men")).\
                                                                sort("productrevenue", ascending=[0])
departmentsSalesDataforMensprod.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataforMensprod", compression="none", header ='true')
departmentsSalesDataforMensprod.show(20,False)

+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
department_name|product_name |avg_sales|min_sales|max_sales|productrevenue|Numberofsales|
+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
Footwear |Nike Men's Free 5.0+ Running Shoe |301.39 |99.99 |499.95 |3667633.2 |12169 |
Golf |Nike Men's Dri-FIT Victory Golf Polo |149.65 |50.0 |250.0 |3147800.0 |21035 |
Apparel |Nike Men's CJ Elite 2 TD Football Cleat |129.99 |129.99 |129.99 |2891757.66 |22246 |
Fan Shop |O'Brien Men's Neoprene Life Vest |149.7 |49.98 |249.9 |2888993.91 |19298 |
Fitness |adidas Men's F10 Messi TRX FG Soccer Cleat |184.69 |59.99 |299.95 |56330.61 |305 |
Fitness |Nike Men's Comfort 2 Slide |135.93 |44.99 |224.95 |44585.09 |328 |
Footwear |Under Armour Men's Compression EV SL Slide |130.63 |44.99 |224.95 |36576.87 |280 |
Golf |Columbia Men's PFG Anchor Tough T-Shirt |91.58 |30.0 |150.0 |27840.0 |304 |
Golf |Nike Men's Deutschland Weltmeister Winners Bl|91.31 |30.0 |150.0 |27210.0 |298 |
Golf |adidas Men's Germany Black Crest Away Tee |74.31 |25.0 |125.0 |21475.0 |289 |
Fitness |Under Armour Men's Tech II T-Shirt |74.52 |24.99 |124.95 |20866.65 |280 |
Footwear |Nike Men's Free TR 5.0 TB Training Shoe |302.91 |99.99 |499.95 |20597.94 |68 |
Outdoors |Hirzl Men's Hybrid Golf Glove |45.93 |14.99 |74.95 |12951.36 |282 |
Golf |Nike Men's Kobe IX Elite Low Basketball Shoe |199.99 |199.99 |199.99 |12599.37 |63 |
Outdoors |Merrell Men's All Out Flash Trail Running Sho|109.99 |109.99 |109.99 |9349.15 |85 |
Fitness |Nike Men's Fingertrap Max Training Shoe |124.99 |124.99 |124.99 |7999.36 |64 |
+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+

###### Problem statement-7:Get monthly sales data from kids and women categories for first two quarters of 2014

In [34]:
SalesDataPerMonth=deptprodordersdfsalesmap.where((to_date(deptprodordersdfsalesmap.order_date) >=lit("2014-01-01")) & 
                                                 (to_date(deptprodordersdfsalesmap.order_date) <=lit("2014-06-30")))

departmentsSalesDataforkidwomen= SalesDataPerMonth.groupBy(substring('order_date',0,7).alias('ordermonth'),'department_name','product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("productrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter(deptprodordersdfsalesmap.product_name.contains("Women")|deptprodordersdfsalesmap.product_name.contains("Kid")).\
                                                                sort('ordermonth',"productrevenue", ascending=[1,0])
departmentsSalesDataforkidwomen.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataforkidwomen", compression="none", header ='true')
departmentsSalesDataforkidwomen.show(30,False)

+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
ordermonth|department_name|product_name |avg_sales|min_sales|max_sales|productrevenue|Numberofsales|
+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
2014-01 |Fan Shop |Diamondback Women's Serene Classic Comfort Bi|299.98 |299.98 |299.98 |341677.23 |1139 |
2014-01 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |208.0 |65.0 |325.0 |4160.0 |20 |
2014-01 |Footwear |Under Armour Women's Micro G Skulpt Running S|158.31 |54.97 |274.85 |3957.84 |25 |
2014-01 |Outdoors |LIJA Women's Mid-Length Panel Golf Shorts |333.33 |100.0 |500.0 |3000.0 |9 |
2014-01 |Footwear |Under Armour Women's Ignite Slide |98.34 |31.99 |159.95 |2655.17 |27 |
2014-01 |Footwear |Nike Women's Tempo Shorts |95.0 |30.0 |150.0 |2280.0 |24 |
2014-01 |Footwear |Under Armour Kids' Mercenary Slide |90.33 |27.99 |139.95 |1987.29 |22 |
2014-01 |Outdoors |Glove It Women's Imperial Golf Glove |63.97 |19.99 |99.95 |1919.04 |30 |
2014-01 |Fitness |Nike Kids' Grade School KD VI Basketball Shoe|271.4 |99.99 |399.96 |1899.81 |7 |
2014-01 |Outdoors |Merrell Women's Grassbow Sport Hiking Shoe |316.63 |199.98 |499.95 |1899.81 |6 |
2014-01 |Outdoors |Glove It Women's Mod Oval 3-Zip Carry All Gol|66.76 |21.99 |109.95 |1869.15 |28 |
2014-01 |Footwear |Under Armour Women's Ignite PIP VI Slide |88.35 |31.99 |159.95 |1855.42 |21 |
2014-01 |Footwear |Brooks Women's Ghost 6 Running Shoe |224.97 |89.99 |359.96 |1799.8 |8 |
2014-01 |Fitness |adidas Kids' F5 Messi FG Soccer Cleat |107.16 |34.99 |174.95 |1714.51 |16 |
2014-01 |Fan Shop |The North Face Women's Recon Backpack |396.0 |198.0 |495.0 |1584.0 |4 |
2014-01 |Footwear |Nike Women's Free 5.0 TR FIT PRT 4 Training S|265.97 |189.98 |379.96 |1329.86 |5 |
2014-01 |Outdoors |Top Flite Women's 2014 XL Hybrid |65.97 |19.99 |99.95 |1319.34 |20 |
2014-01 |Fitness |Nike Women's Legend V-Neck T-Shirt |65.79 |25.0 |125.0 |1250.0 |19 |
2014-01 |Outdoors |Glove It Women's Mod Oval Golf Glove |54.26 |19.99 |99.95 |1139.43 |21 |
2014-01 |Outdoors |Hirzl Women's Soffft Flex Golf Glove |51.4 |17.99 |89.95 |1079.4 |21 |
2014-01 |Outdoors |Hirzl Women's Hybrid Golf Glove |36.09 |14.99 |74.95 |974.35 |27 |
2014-01 |Outdoors |TaylorMade Women's RBZ SL Rescue |129.99 |129.99 |129.99 |909.93 |7 |
2014-01 |Outdoors |Cleveland Golf Women's 588 RTX CB Satin Chrom|119.99 |119.99 |119.99 |839.93 |7 |
2014-01 |Outdoors |LIJA Women's Argyle Golf Polo |200.0 |80.0 |320.0 |800.0 |4 |
2014-01 |Outdoors |LIJA Women's Button Golf Dress |108.0 |108.0 |108.0 |756.0 |7 |
2014-01 |Outdoors |Merrell Women's Siren Mid Waterproof Hiking B|134.99 |134.99 |134.99 |674.95 |5 |
2014-01 |Outdoors |Merrell Women's Grassbow Sport Waterproof Hik|129.99 |129.99 |129.99 |649.95 |5 |
2014-02 |Fan Shop |Diamondback Women's Serene Classic Comfort Bi|299.98 |299.98 |299.98 |332977.81 |1110 |
2014-02 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |207.19 |65.0 |325.0 |6630.0 |32 |
2014-02 |Footwear |Under Armour Women's Micro G Skulpt Running S|162.41 |54.97 |274.85 |3573.05 |22 |
+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
only showing top 30 rows

###### Problem statement-8:Get monthly sales data of TOP10 revenue generating Nike and adidas products with rankings

In [36]:
departSalesDatafortop10 = deptprodordersdfsalesmap.groupBy(substring('order_date',0,7).alias('ordermonth'),'department_name','product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("productrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter(deptprodordersdfsalesmap.product_name.contains("Nike")|deptprodordersdfsalesmap.product_name.contains("adidas")).\
                                                                sort('ordermonth',"productrevenue", ascending=[1,0])

window = Window.partitionBy(departSalesDatafortop10['ordermonth']).orderBy(departSalesDatafortop10['productrevenue'].desc())
top10deptdataeachmonth=departSalesDatafortop10.select('ordermonth','department_name','productrevenue').withColumn('rank', rank().over(window).alias('rank'))
top10rankeddeptdataeachmonth = top10deptdataeachmonth.filter(col('rank')<=10).sort('ordermonth','productrevenue',ascending=[1,0])

top10rankeddeptdataeachmonth.coalesce(1).write.csv("/FileStore/tables/top10rankeddeptdataeachmonth", compression="none", header ='true')
top10rankeddeptdataeachmonth.show(30,False)

+----------+---------------+--------------+----+
ordermonth|department_name|productrevenue|rank|
+----------+---------------+--------------+----+
2013-07 |Footwear |78292.17 |1 |
2013-07 |Golf |67100.0 |2 |
2013-07 |Apparel |59925.39 |3 |
2013-07 |Fitness |1484.67 |4 |
2013-07 |Golf |1050.0 |5 |
2013-07 |Footwear |840.0 |6 |
2013-07 |Fitness |725.0 |7 |
2013-07 |Fitness |659.89 |8 |
2013-07 |Golf |630.0 |9 |
2013-07 |Fitness |528.0 |10 |
2013-08 |Footwear |292870.71 |1 |
2013-08 |Golf |261150.0 |2 |
2013-08 |Apparel |231902.17 |3 |
2013-08 |Golf |6790.0 |4 |
2013-08 |Fitness |6358.94 |5 |
2013-08 |Fitness |3644.19 |6 |
2013-08 |Footwear |2340.0 |7 |
2013-08 |Footwear |2099.79 |8 |
2013-08 |Golf |2075.0 |9 |
2013-08 |Fitness |1958.0 |10 |
2013-09 |Footwear |340165.98 |1 |
2013-09 |Golf |264900.0 |2 |
2013-09 |Apparel |245421.13 |3 |
2013-09 |Fitness |5639.06 |4 |
2013-09 |Golf |5390.0 |5 |
2013-09 |Fitness |2789.38 |6 |
2013-09 |Footwear |2760.0 |7 |
2013-09 |Fitness |2599.74 |8 |
2013-09 |Golf |2370.0 |9 |
2013-09 |Fitness |2204.37 |10 |
+----------+---------------+--------------+----+
only showing top 30 rows

###### Problem statement-9:Get quarterly sales data of golf products for 2013 final quarter

In [38]:
SalesDataPerMonth=deptprodordersdfsalesmap.where((to_date(deptprodordersdfsalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(deptprodordersdfsalesmap.order_date) <=lit("2013-12-31")))

departmentsSalesDataforkidwomen= SalesDataPerMonth.groupBy(substring('order_date',0,7).alias('ordermonth'),'department_name','product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("productrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter(deptprodordersdfsalesmap.product_name.contains("Golf")).\
                                                                sort('ordermonth',"productrevenue", ascending=[1,0])
departmentsSalesDataforkidwomen.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataforkidwomen", compression="none", header ='true')
departmentsSalesDataforkidwomen.show(30,False)

+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
ordermonth|department_name|product_name |avg_sales|min_sales|max_sales|productrevenue|Numberofsales|
+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
2013-10 |Golf |Nike Men's Dri-FIT Victory Golf Polo |150.55 |50.0 |250.0 |247050.0 |1641 |
2013-10 |Outdoors |Titleist Pro V1x Golf Balls |134.97 |47.99 |239.95 |4319.1 |32 |
2013-10 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |198.1 |65.0 |325.0 |4160.0 |21 |
2013-10 |Outdoors |Titleist Pro V1x High Numbers Golf Balls |154.63 |47.99 |239.95 |2783.42 |18 |
2013-10 |Outdoors |Team Golf Texas Longhorns Putter Grip |71.13 |24.99 |124.95 |1849.26 |26 |
2013-10 |Outdoors |Team Golf Pittsburgh Steelers Putter Grip |69.2 |24.99 |124.95 |1799.28 |26 |
2013-10 |Outdoors |LIJA Women's Argyle Golf Polo |220.0 |80.0 |400.0 |1760.0 |8 |
2013-10 |Outdoors |Team Golf Tennessee Volunteers Putter Grip |76.06 |24.99 |124.95 |1749.3 |23 |
2013-10 |Outdoors |Team Golf St. Louis Cardinals Putter Grip |70.8 |24.99 |124.95 |1699.32 |24 |
2013-10 |Outdoors |Team Golf San Francisco Giants Putter Grip |80.23 |24.99 |124.95 |1524.39 |19 |
2013-10 |Outdoors |Garmin Approach S4 Golf GPS Watch |299.99 |299.99 |299.99 |1499.95 |5 |
2013-10 |Outdoors |Cleveland Golf Collegiate My Custom Wedge 588|209.99 |209.99 |209.99 |1469.93 |7 |
2013-10 |Outdoors |Ogio Race Golf Shoes |169.99 |169.99 |169.99 |1359.92 |8 |
2013-10 |Outdoors |Glove It Women's Imperial Golf Glove |55.81 |19.99 |99.95 |1339.33 |24 |
2013-10 |Outdoors |Glove It Women's Mod Oval Golf Glove |67.33 |19.99 |99.95 |1279.36 |19 |
2013-10 |Outdoors |Garmin Approach S3 Golf GPS Watch |249.99 |249.99 |249.99 |1249.95 |5 |
2013-10 |Outdoors |Team Golf New England Patriots Putter Grip |76.53 |24.99 |124.95 |1224.51 |16 |
2013-10 |Outdoors |Glove It Urban Brick Golf Towel |49.97 |15.99 |79.95 |1199.25 |24 |
2013-10 |Outdoors |Hirzl Women's Soffft Flex Golf Glove |46.93 |17.99 |89.95 |1079.4 |23 |
2013-10 |Outdoors |Glove It Imperial Golf Towel |51.17 |15.99 |79.95 |1023.36 |20 |
2013-10 |Outdoors |Hirzl Women's Hybrid Golf Glove |43.47 |14.99 |74.95 |869.42 |20 |
2013-10 |Outdoors |MDGolf Pittsburgh Penguins Putter |159.98 |79.99 |399.95 |799.9 |5 |
2013-10 |Outdoors |Hirzl Men's Hybrid Golf Glove |45.85 |14.99 |74.95 |779.48 |17 |
2013-10 |Outdoors |GolfBuddy VT3 GPS Watch |199.99 |199.99 |199.99 |599.97 |3 |
2013-10 |Outdoors |LIJA Women's Mid-Length Panel Golf Shorts |250.0 |200.0 |300.0 |500.0 |2 |
2013-10 |Outdoors |Cleveland Golf Women's 588 RTX CB Satin Chrom|119.99 |119.99 |119.99 |479.96 |4 |
2013-10 |Outdoors |LIJA Women's Button Golf Dress |108.0 |108.0 |108.0 |108.0 |1 |
2013-11 |Golf |Nike Men's Dri-FIT Victory Golf Polo |147.45 |50.0 |250.0 |291500.0 |1977 |
2013-11 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |235.93 |65.0 |325.0 |6370.0 |27 |
2013-11 |Outdoors |Titleist Pro V1x High Numbers Golf Balls |137.43 |47.99 |239.95 |3023.37 |22 |
+----------+---------------+---------------------------------------------+---------+---------+---------+--------------+-------------+
only showing top 30 rows

###### Problem statement-10:Get sales data of TOP3 Apparel products for each year

In [40]:
apparelSalesDatatop3peryear = deptprodordersdfsalesmap.groupBy(substring('order_date',0,4).alias('orderyear'),'department_name','product_name').\
                                                                agg(round(avg('order_item_subtotal'),2).alias("avg_sales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("productrevenue"),
                                                                count('order_item_subtotal').alias("Numberofsales")).\
                                                                filter(deptprodordersdfsalesmap.department_name=="Apparel").\
                                                                sort('orderyear',"productrevenue", ascending=[1,0])

window = Window.partitionBy(apparelSalesDatatop3peryear['orderyear']).orderBy(apparelSalesDatatop3peryear['productrevenue'].desc())
top3proddataeachmonth=apparelSalesDatatop3peryear.select('orderyear','department_name','product_name','Numberofsales','avg_sales','productrevenue').withColumn('rank', rank().over(window).alias('rank'))
top3rankedproddataeachmonth = top5proddataeachmonth.filter(col('rank')<=5).sort('orderyear','productrevenue',ascending=[1,0])
top3rankedproddataeachmonth.coalesce(1).write.csv("/FileStore/tables/top3rankedproddataeachmonth", compression="none", header ='true')
top3rankedproddataeachmonth.show(30,False)

+---------+---------------+---------------------------------------+-------------+---------+--------------+----+
orderyear|department_name|product_name |Numberofsales|avg_sales|productrevenue|rank|
+---------+---------------+---------------------------------------+-------------+---------+--------------+----+
2013 |Apparel |Perfect Fitness Perfect Rip Deck |10797 |179.9 |1942416.26 |1 |
2013 |Apparel |Nike Men's CJ Elite 2 TD Football Cleat|9848 |129.99 |1280141.57 |2 |
2013 |Apparel |Total Gym 1400 |20 |299.99 |5999.8 |3 |
2014 |Apparel |Perfect Fitness Perfect Rip Deck |13718 |180.69 |2478726.88 |1 |
2014 |Apparel |Nike Men's CJ Elite 2 TD Football Cleat|12398 |129.99 |1611616.09 |2 |
2014 |Apparel |Total Gym 1400 |16 |299.99 |4799.84 |3 |
+---------+---------------+---------------------------------------+-------------+---------+--------------+----+